In [1]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.datasets import load_digits
from sklearn.datasets import load_boston

In [2]:
def to_label(data, target, percentile):
    frac = percentile / 100.0
    part_val = data[target].quantile(frac)
    data[target] = [1 if d > part_val else 0 for d in data[target]]
    return data

In [3]:
#digits = load_digits()
boston = load_boston()
#data = pd.DataFrame(digits.data)
data = pd.DataFrame(boston.data, columns=boston.feature_names)
#label = 'class'
label = 'HomeVal50'
#data[label] = digits.target
data[label] = boston.target
to_label(data, label, 50)
data.head(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,HomeVal50
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,1
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,1
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,1
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,1
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,1


In [21]:
t = np.array(data[label])
X = data.drop(label, axis=1)

In [22]:
t.shape

(506,)

In [23]:
N = X.shape[0]
D = X.shape[1]
print(N,D)

506 13


In [24]:
# add a column of ones to X
ones = np.array([[1]*N]).T
phi = np.concatenate((ones,X), axis=1)

In [25]:
w = np.random.randn(D+1)
a = np.dot(phi,w)

In [42]:
w.shape

(14,)

In [28]:
def sigmoid(a):
    return 1/(1 + np.exp(-a))

In [12]:
def softmax(a):
    e = np.exp(a - np.max(a))
    #e = np.exp(a)
    if e.ndim == 1:
        return e / np.sum(e, axis=0)
    else:
        return e / np.array([np.sum(e, axis=1)]).T

In [29]:
p = sigmoid(a)
grad = np.dot((p - t), phi)

In [30]:
grad

array([-1.12416556e+00,  4.66794033e+02,  1.44959297e+02,  5.62449277e+02,
       -1.30442485e+01,  6.08133906e+00, -7.98554726e+01, -7.87974473e+02,
        3.13716323e+01,  1.17897853e+03,  2.55109264e+04,  2.58063358e+02,
       -3.91951230e+03,  6.23198798e+02])

In [44]:
R = np.diag(p * (1 - p))

In [55]:
z = np.dot(phi, w) - (np.dot((p-t), np.linalg.pinv(R)))

In [63]:
np.linalg.inv(np.dot(np.dot(phi.T,R),phi)).dot(phi.T).dot(R).dot(z).shape

(14,)

In [62]:
z.shape

(506,)

In [64]:
np.zeros(D+1)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [78]:
np.dot(phi.T,phi).dot(w) - np.dot(phi.T,t)

array([  -1246.72723329,  -24714.62879249,  191930.84941391,
        -54761.6632841 ,    -576.49448914,   -1611.1190785 ,
         -5129.52313752, -547368.96761597,   17073.89302026,
         25372.26449013,  176888.73929845,  -21246.03431071,
       -112504.30689689,  -86996.71524635])

In [79]:
(p-t).dot(phi)

array([-1.12416556e+00,  4.66794033e+02,  1.44959297e+02,  5.62449277e+02,
       -1.30442485e+01,  6.08133906e+00, -7.98554726e+01, -7.87974473e+02,
        3.13716323e+01,  1.17897853e+03,  2.55109264e+04,  2.58063358e+02,
       -3.91951230e+03,  6.23198798e+02])

In [89]:
t*np.log(2*p) + (1-t)*np.log(1-2*p)

C:\Users\ptngu\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.


array([-3.48285953e+00, -4.48306987e+01, -1.17667564e+01,             nan,
       -1.20393016e+00, -7.27727123e+00, -1.30403814e+01, -6.62382131e+01,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                   nan, -3.18025397e-02,  0.00000000e+00,             nan,
                   nan,  0.00000000e+00,             nan, -1.15204990e-09,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -1.11363047e-08,
       -3.35900669e-06,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
                   nan,             nan,             nan,  0.00000000e+00,
        0.00000000e+00, -6.27276009e-14, -2.13910360e+00, -4.93312059e-09,
                   nan,  

In [98]:
def safe_ln(p):
    for x in p:
        if x <= 0:
            return 0
        return np.log(x)

In [100]:
-np.sum(t*safe_ln(p) + (1-t)*safe_ln(1-p))

1047.964277237255